In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emi-sections-data/Extracted_sections_baggage_stacked.csv
/kaggle/input/emi-ada3-copilot/clara_evaluation.csv
/kaggle/input/emi-ada3-copilot/categorized_ada3large.csv
/kaggle/input/emi-ada2/categorized_ada2.csv


In [2]:
ada3 = pd.read_csv('/kaggle/input/emi-ada3-copilot/categorized_ada3large.csv')
copilot = pd.read_csv('/kaggle/input/emi-ada3-copilot/clara_evaluation.csv')
sections = pd.read_csv('/kaggle/input/emi-sections-data/Extracted_sections_baggage_stacked.csv')
ada2 = pd.read_csv('/kaggle/input/emi-ada2/categorized_ada2.csv')

updates = {
    "da4dd92b-69b7-4476-944f-5a6906da81bb": 178,
    "61309168-52bc-427b-a971-36c02f4c23a8": 3
}



In [3]:
ada3.drop(columns=['answer','contexts'], errors='ignore', inplace=True)
ada3.head()


,id,date,question,area,question_embedding,new_context,new_answer
0,1757d8aa-f045-439a-ae33-5612fab06d28,2025-08-19T10:33:03.829000+0000,What is the smart baggage acceptance on Emirates?,checkin,"[0.003234968287870288, -0.005037979688495398, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...",Smart baggage acceptance on Emirates requires ...
1,e3204adf-967a-4d75-9632-96d72ebd1426,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.7.pdf',...","Apologies, the **Concourse SOP does not includ..."
2,ab1c0223-a197-4d0b-915c-f46c483aeb91,2025-08-19T09:16:08.223000+0000,passenger with passport yemen can enter to cai...,checkin,"[-0.014634286053478718, -0.0014922679401934147...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
3,0c2d95ba-ef22-4de6-ada8-469b34543e3e,2025-08-19T08:45:42.950000+0000,How can I insert the new flight number?,checkin,"[-0.034327492117881775, 0.006717535201460123, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
4,42ff4dc8-c09c-4f31-8a23-81ea77023c7e,2025-08-19T08:40:56.117000+0000,Should we tag the bags for staff ticket either...,checkin,"[0.002173528540879488, -0.007075493223965168, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."


In [4]:
copilot.head()

,id,question,area,Answer,Classification,Page Reference
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206


In [5]:
ada3['id'] = ada3['id'].astype(str).str.strip()
copilot['id'] = copilot['id'].astype(str).str.strip()

cop_pref = copilot.rename(columns=lambda c: f'copilot_{c}' if c not in {'id', 'question'} else c)
ada_pref = ada3.rename(columns=lambda c: f'ada3_{c}'    if c not in {'id', 'question'} else c)

merged = cop_pref.merge(ada_pref, on='id', how='left', validate='many_to_one')
merged.head()

,id,question_x,copilot_area,copilot_Answer,copilot_Classification,copilot_Page Reference,ada3_date,question_y,ada3_area,ada3_question_embedding,ada3_new_context,ada3_new_answer
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240,2025-07-31T06:34:09.658000+0000,What are the rules for zamzam in chapter 8?,checkin,"[-0.010032432153820992, -0.02128632366657257, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174,2025-08-08T03:51:14.352000+0000,What is the meaning of YPTA-Young Passenger Tr...,checkin,"[0.006769979372620583, -0.02541240118443966, 0...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185,2025-07-31T06:59:08.984000+0000,What should I have ready when I call the medic...,checkin,"[0.009598040953278542, -0.0035161797422915697,...","[Document(filename='/Check-in SOP v45.7.pdf', ...",You should have the following details ready wh...
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262,2025-08-14T03:41:04.701000+0000,can thermometers carried by habd?,checkin,"[-0.0037370906211435795, 0.013638687320053577,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.7.pdf',...","Apologies, the **Concourse SOP does not includ..."


In [6]:

import re
import numpy as np
import pandas as pd

def to_avg_or_num(val):
    if pd.isna(val):
        return np.nan
    
    if isinstance(val, (int, float, np.integer, np.floating)):
        return float(val)
    
    s = str(val).strip()

    m_range = re.fullmatch(r'(\d+(?:\.\d+)?)\s*[-–—]\s*(\d+(?:\.\d+)?)', s)
    if m_range:
        a = float(m_range.group(1))
        b = float(m_range.group(2))
        return (a + b) / 2.0

    m_single = re.fullmatch(r'\d+(?:\.\d+)?', s)
    if m_single:
        return float(s)

    return np.nan

col = 'copilot_Page Reference' 

merged['copilot_page_reference_num'] = (
    merged[col].apply(to_avg_or_num).astype('Float64') 
)


merged['ada3_page_num'] = (
    merged['ada3_new_context']
        .astype(str)
        .str.extract(r'page\s*=\s*(\d+(?:\.\d+)?)', expand=False)
        .astype('Float64')  
)


merged['ada3_chunk_id'] = (
    merged['ada3_new_context']
        .astype(str)
        .str.extract(r"chunk_id\s*=\s*['\"]?([^,'\"\]\s)]+)", expand=False)  
        .astype('string')
)


merged['ada3_page_num'] = merged['ada3_page_num'].round().astype('Int64')

merged.head()


,id,question_x,copilot_area,copilot_Answer,copilot_Classification,copilot_Page Reference,ada3_date,question_y,ada3_area,ada3_question_embedding,ada3_new_context,ada3_new_answer,copilot_page_reference_num,ada3_page_num,ada3_chunk_id
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240,2025-07-31T06:34:09.658000+0000,What are the rules for zamzam in chapter 8?,checkin,"[-0.010032432153820992, -0.02128632366657257, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",239.5,251,C60
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174,2025-08-08T03:51:14.352000+0000,What is the meaning of YPTA-Young Passenger Tr...,checkin,"[0.006769979372620583, -0.02541240118443966, 0...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",174.0,182,C38
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185,2025-07-31T06:59:08.984000+0000,What should I have ready when I call the medic...,checkin,"[0.009598040953278542, -0.0035161797422915697,...","[Document(filename='/Check-in SOP v45.7.pdf', ...",You should have the following details ready wh...,185.0,6,C3
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262,2025-08-14T03:41:04.701000+0000,can thermometers carried by habd?,checkin,"[-0.0037370906211435795, 0.013638687320053577,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",262.0,261,C63
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.7.pdf',...","Apologies, the **Concourse SOP does not includ...",206.0,26,C26


In [7]:
import pandas as pd

cop_col = 'copilot_page_reference_num' 
ada_col = 'ada3_page_num'

merged[cop_col] = pd.to_numeric(merged[cop_col], errors='coerce')
merged[ada_col] = pd.to_numeric(merged[ada_col], errors='coerce')

diff = (merged[ada_col] - merged[cop_col]).abs()

merged['within_5'] = diff.le(5).where(diff.notna()).astype('boolean')
merged['within_10'] = diff.le(10).where(diff.notna()).astype('boolean')

merged['page_diff'] = diff


print(merged[['id', cop_col, ada_col, 'page_diff', 'within_5', 'within_10']].head())

print('within_5 counts:\n', merged['within_5'].value_counts(dropna=False))
print('within_10 counts:\n', merged['within_10'].value_counts(dropna=False))



                                     id  copilot_page_reference_num  \
0  da34a043-7b9b-4b0b-88fb-4e503daa80a9                       239.5   
1  a0a387ba-5f5c-4f2e-8474-f3219b52331c                       174.0   
2  2b97081d-f77e-4aea-9997-1f9e6d673785                       185.0   
3  47cebc3a-9e4f-41a8-943a-f5b5c791c9d7                       262.0   
4  e3204adf-967a-4d75-9632-96d72ebd1426                       206.0   

   ada3_page_num  page_diff  within_5  within_10  
0            251       11.5     False      False  
1            182        8.0     False       True  
2              6      179.0     False      False  
3            261        1.0      True       True  
4             26      180.0     False      False  
within_5 counts:
 within_5
False    131
True      21
<NA>      14
Name: count, dtype: Int64
within_10 counts:
 within_10
False    105
True      47
<NA>      14
Name: count, dtype: Int64


In [8]:
merged.to_csv('EMI_comparison.csv', index=False)

In [9]:
import pandas as pd


merged['copilot_area'] = merged['copilot_area'].str.strip().str.lower()
sections['area'] = sections['area'].str.strip().str.lower()

sections['page'] = pd.to_numeric(sections['page'], errors='coerce')
sections['end_page'] = pd.to_numeric(sections['end_page'], errors='coerce').fillna(sections['page'])
merged['page_num'] = pd.to_numeric(merged['copilot_page_reference_num'], errors='coerce')

area_map = {}
for _, row in sections.iterrows():
    area_map.setdefault(row['area'], []).append((row['page'], row['end_page'], row['chunk_id']))

# Function to find section
def find_section(area, page):
    if pd.isna(page) or area not in area_map:
        return None
    for start, end, chunk in area_map[area]:
        if start <= page <= end:
            return chunk
    return None

# Apply mapping
merged['copilot_section'] = merged.apply(lambda r: find_section(r['copilot_area'], r['page_num']), axis=1)

# Save result
merged.to_csv("EMI_comparison_with_section.csv", index=False)
print("Done! File saved as EMI_comparison_with_section.csv")
merged.head()


Done! File saved as EMI_comparison_with_section.csv


,id,question_x,copilot_area,copilot_Answer,copilot_Classification,copilot_Page Reference,ada3_date,question_y,ada3_area,ada3_question_embedding,ada3_new_context,ada3_new_answer,copilot_page_reference_num,ada3_page_num,ada3_chunk_id,within_5,within_10,page_diff,page_num,copilot_section
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240,2025-07-31T06:34:09.658000+0000,What are the rules for zamzam in chapter 8?,checkin,"[-0.010032432153820992, -0.02128632366657257, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",239.5,251,C60,False,False,11.5,239.5,C55
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174,2025-08-08T03:51:14.352000+0000,What is the meaning of YPTA-Young Passenger Tr...,checkin,"[0.006769979372620583, -0.02541240118443966, 0...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",174.0,182,C38,False,True,8.0,174.0,C34
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185,2025-07-31T06:59:08.984000+0000,What should I have ready when I call the medic...,checkin,"[0.009598040953278542, -0.0035161797422915697,...","[Document(filename='/Check-in SOP v45.7.pdf', ...",You should have the following details ready wh...,185.0,6,C3,False,False,179.0,185.0,C39
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262,2025-08-14T03:41:04.701000+0000,can thermometers carried by habd?,checkin,"[-0.0037370906211435795, 0.013638687320053577,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",262.0,261,C63,True,True,1.0,262.0,C63
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.7.pdf',...","Apologies, the **Concourse SOP does not includ...",206.0,26,C26,False,False,180.0,206.0,None


In [10]:
ada2['id'] = ada2['id'].astype(str).str.strip()

ada_pref_2 = ada2.rename(columns=lambda c: f'ada2_{c}'    if c not in {'id', 'question'} else c)
ada_pref_2 = ada_pref_2.drop('question', axis = 1)

merged = merged.merge(ada_pref_2, on='id', how='left', validate='many_to_one')
merged.head()

,id,question_x,copilot_area,copilot_Answer,copilot_Classification,copilot_Page Reference,ada3_date,question_y,ada3_area,ada3_question_embedding,...,page_diff,page_num,copilot_section,ada2_date,ada2_contexts,ada2_answer,ada2_area,ada2_question_embedding,ada2_new_context,ada2_new_answer
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240,2025-07-31T06:34:09.658000+0000,What are the rules for zamzam in chapter 8?,checkin,"[-0.010032432153820992, -0.02128632366657257, ...",...,11.5,239.5,C55,2025-07-31T06:34:09.658000+0000,"[""Acceptance Policy: • Accept only one jerry c...","Apologies, the **Checkin SOP does not include ...",checkin,"[-0.010032432153820992, -0.02128632366657257, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174,2025-08-08T03:51:14.352000+0000,What is the meaning of YPTA-Young Passenger Tr...,checkin,"[0.006769979372620583, -0.02541240118443966, 0...",...,8.0,174.0,C34,2025-08-08T03:51:14.352000+0000,['Check-in Agent 1. Direct the Unaccompanied M...,"Apologies, the **Checkin SOP does not include ...",checkin,"[0.006769979372620583, -0.02541240118443966, 0...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185,2025-07-31T06:59:08.984000+0000,What should I have ready when I call the medic...,checkin,"[0.009598040953278542, -0.0035161797422915697,...",...,179.0,185.0,C39,2025-07-31T06:59:08.984000+0000,"[""All Staff Reduced mobility is whose mobility...","Apologies, the **Checkin SOP does not include ...",checkin,"[0.009598040953278542, -0.0035161797422915697,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","When calling the medical team, you should have..."
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262,2025-08-14T03:41:04.701000+0000,can thermometers carried by habd?,checkin,"[-0.0037370906211435795, 0.013638687320053577,...",...,1.0,262.0,C63,2025-08-14T03:41:04.701000+0000,['All Staff Whether a lithium battery can be c...,"Apologies, the **Checkin SOP does not include ...",checkin,"[-0.0037370906211435795, 0.013638687320053577,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ..."
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...",...,180.0,206.0,None,2025-08-19T09:38:54.594000+0000,['Airside Self Service Kiosks – Set up next to...,"Apologies, the **Concourse SOP does not includ...",concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.8.pdf',...","Apologies, the **Concourse SOP does not includ..."


In [11]:
merged['ada2_chunk_id'] = (
    merged['ada2_new_context']
        .astype(str)
        .str.extract(r"chunk_id\s*=\s*['\"]?([^,'\"\]\s)]+)", expand=False)  
        .astype('string')
)

merged.head()


,id,question_x,copilot_area,copilot_Answer,copilot_Classification,copilot_Page Reference,ada3_date,question_y,ada3_area,ada3_question_embedding,...,page_num,copilot_section,ada2_date,ada2_contexts,ada2_answer,ada2_area,ada2_question_embedding,ada2_new_context,ada2_new_answer,ada2_chunk_id
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240,2025-07-31T06:34:09.658000+0000,What are the rules for zamzam in chapter 8?,checkin,"[-0.010032432153820992, -0.02128632366657257, ...",...,239.5,C55,2025-07-31T06:34:09.658000+0000,"[""Acceptance Policy: • Accept only one jerry c...","Apologies, the **Checkin SOP does not include ...",checkin,"[-0.010032432153820992, -0.02128632366657257, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",C60
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174,2025-08-08T03:51:14.352000+0000,What is the meaning of YPTA-Young Passenger Tr...,checkin,"[0.006769979372620583, -0.02541240118443966, 0...",...,174.0,C34,2025-08-08T03:51:14.352000+0000,['Check-in Agent 1. Direct the Unaccompanied M...,"Apologies, the **Checkin SOP does not include ...",checkin,"[0.006769979372620583, -0.02541240118443966, 0...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",C24
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185,2025-07-31T06:59:08.984000+0000,What should I have ready when I call the medic...,checkin,"[0.009598040953278542, -0.0035161797422915697,...",...,185.0,C39,2025-07-31T06:59:08.984000+0000,"[""All Staff Reduced mobility is whose mobility...","Apologies, the **Checkin SOP does not include ...",checkin,"[0.009598040953278542, -0.0035161797422915697,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","When calling the medical team, you should have...",C43
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262,2025-08-14T03:41:04.701000+0000,can thermometers carried by habd?,checkin,"[-0.0037370906211435795, 0.013638687320053577,...",...,262.0,C63,2025-08-14T03:41:04.701000+0000,['All Staff Whether a lithium battery can be c...,"Apologies, the **Checkin SOP does not include ...",checkin,"[-0.0037370906211435795, 0.013638687320053577,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",C63
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...",...,206.0,None,2025-08-19T09:38:54.594000+0000,['Airside Self Service Kiosks – Set up next to...,"Apologies, the **Concourse SOP does not includ...",concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.8.pdf',...","Apologies, the **Concourse SOP does not includ...",C68


In [12]:
cols_to_drop = ['ada2_contexts_x', 'ada2_answer_x', 'ada2_new_answer_x', 'ada2_new_context_x', 'ada2_question_embedding_y', 'ada2_question_embedding_x' ]
existing_cols = [col for col in cols_to_drop if col in merged.columns]
merged.drop(columns=existing_cols, inplace=True)

merged.head()


,id,question_x,copilot_area,copilot_Answer,copilot_Classification,copilot_Page Reference,ada3_date,question_y,ada3_area,ada3_question_embedding,...,page_num,copilot_section,ada2_date,ada2_contexts,ada2_answer,ada2_area,ada2_question_embedding,ada2_new_context,ada2_new_answer,ada2_chunk_id
0,da34a043-7b9b-4b0b-88fb-4e503daa80a9,What are the rules for zamzam in chapter 8?,checkin,Zamzam water is carried free of charge.\nOnly ...,Response received and verified,239-240,2025-07-31T06:34:09.658000+0000,What are the rules for zamzam in chapter 8?,checkin,"[-0.010032432153820992, -0.02128632366657257, ...",...,239.5,C55,2025-07-31T06:34:09.658000+0000,"[""Acceptance Policy: • Accept only one jerry c...","Apologies, the **Checkin SOP does not include ...",checkin,"[-0.010032432153820992, -0.02128632366657257, ...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",C60
1,a0a387ba-5f5c-4f2e-8474-f3219b52331c,What is the meaning of YPTA-Young Passenger Tr...,checkin,Young Passengers (YP) are those traveling alon...,Response received and verified,174,2025-08-08T03:51:14.352000+0000,What is the meaning of YPTA-Young Passenger Tr...,checkin,"[0.006769979372620583, -0.02541240118443966, 0...",...,174.0,C34,2025-08-08T03:51:14.352000+0000,['Check-in Agent 1. Direct the Unaccompanied M...,"Apologies, the **Checkin SOP does not include ...",checkin,"[0.006769979372620583, -0.02541240118443966, 0...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",C24
2,2b97081d-f77e-4aea-9997-1f9e6d673785,What should I have ready when I call the medic...,checkin,You should be ready with: Medical certificate ...,Response received and verified,185,2025-07-31T06:59:08.984000+0000,What should I have ready when I call the medic...,checkin,"[0.009598040953278542, -0.0035161797422915697,...",...,185.0,C39,2025-07-31T06:59:08.984000+0000,"[""All Staff Reduced mobility is whose mobility...","Apologies, the **Checkin SOP does not include ...",checkin,"[0.009598040953278542, -0.0035161797422915697,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","When calling the medical team, you should have...",C43
3,47cebc3a-9e4f-41a8-943a-f5b5c791c9d7,can thermometers carried by habd?,checkin,Yes. Thermometers are not explicitly restricte...,Response received and verified,262,2025-08-14T03:41:04.701000+0000,can thermometers carried by habd?,checkin,"[-0.0037370906211435795, 0.013638687320053577,...",...,262.0,C63,2025-08-14T03:41:04.701000+0000,['All Staff Whether a lithium battery can be c...,"Apologies, the **Checkin SOP does not include ...",checkin,"[-0.0037370906211435795, 0.013638687320053577,...","[Document(filename='/Check-in SOP v45.7.pdf', ...","Apologies, the **Checkin SOP does not include ...",C63
4,e3204adf-967a-4d75-9632-96d72ebd1426,can we offer stpc to miscon passengrs?,concourse,Yes. STPC (Stopover Paid by Carrier) can be of...,Response received and verified,206,2025-08-19T09:38:54.594000+0000,can we offer stpc to miscon passengrs?,concourse,"[0.008406477980315685, -0.0014252571854740381,...",...,206.0,None,2025-08-19T09:38:54.594000+0000,['Airside Self Service Kiosks – Set up next to...,"Apologies, the **Concourse SOP does not includ...",concourse,"[0.008406477980315685, -0.0014252571854740381,...","[Document(filename='/Concourse SOP V29.8.pdf',...","Apologies, the **Concourse SOP does not includ...",C68


In [13]:
ada_3large_howmany = merged[~merged['ada3_new_answer'].str.contains("Apologies", na=False)]
ada_2large_howmany = merged[~merged['ada2_new_answer'].str.contains("Apologies", na=False)]

print(merged.shape)
print(ada_3large_howmany.shape)
print(ada_2large_howmany.shape)

(166, 28)
(30, 28)
(25, 28)
